Wczytujemy surowa baze i filtrujemy dla Polski


In [4]:

import pandas as pd

# Zakładamy, że masz pełny plik danych w formacie CSV i znasz jego ścieżkę
full_file_path = 'zrodla/Database to calculate popularity.csv' # Zastąp tym ścieżką do pełnego pliku

# Wczytanie pełnego pliku CSV do DataFrame, przyjmując, że separator to średnik
full_df = pd.read_csv(full_file_path, sep=',', low_memory=False)

# Filtracja danych dla Polski
full_df_poland = full_df[full_df['country'] == 'Poland']

# Zapis przefiltrowanych danych do nowego pliku CSV
output_full_csv_path = 'poland/Database to calculate popularity Poland.csv'
full_df_poland.to_csv(output_full_csv_path, index=False)

print("Zapisano przefiltrowane dane do", output_full_csv_path)

Zapisano przefiltrowane dane do poland/Database to calculate popularity Poland.csv


Wczytujemy wynikowa baze i filtrujemy dla Polski


In [5]:
import pandas as pd

# Zakładamy, że masz pełny plik danych w formacie CSV i znasz jego ścieżkę
full_file_path = 'zrodla/Final database.csv' # Zastąp tym ścieżką do pełnego pliku

# Wczytanie pełnego pliku CSV do DataFrame, przyjmując, że separator to średnik
full_df = pd.read_csv(full_file_path, sep=',', low_memory=False)

# Filtracja danych dla Polski
full_df_poland = full_df[full_df['Country'] == 'Poland']

# Zapis przefiltrowanych danych do nowego pliku CSV
output_full_csv_path = 'poland/Final database Poland.csv'
full_df_poland.to_csv(output_full_csv_path, index=False)

print("Zapisano przefiltrowane dane do", output_full_csv_path)

Zapisano przefiltrowane dane do poland/Final database Poland.csv


W pliku wynikowym w kolumnie Popu_max szukamy ktore utowry w Polsce doszly do pierszego miejsca na liscie i zapisujemy uri tych utowrow do tymczasowego pliku 'jedynki.csv'

In [6]:
import pandas as pd

# Załaduj pełny plik 'Final database Poland.csv' (zakładając, że plik znajduje się w odpowiednim katalogu)
file_path = 'poland/Final database Poland.csv'
final_db_poland = pd.read_csv(file_path, sep=',', low_memory=False)

# Sortuj dane według kolumny 'Popu_max' w porządku rosnącym
sorted_data = final_db_poland.sort_values(by='Popu_max', ascending=True)

# Wybierz rekordy, gdzie 'Popu_max' wynosi 1
top_tracks = sorted_data[sorted_data['Popu_max'] == 1]

# Wyodrębnij kolumnę 'Uri' i zapisz do nowego DataFrame
top_tracks_uris = top_tracks[['Uri']].copy()

# Zapisz wynikowy DataFrame do pliku CSV
output_file_path = 'tmp/jedynki.csv'
top_tracks_uris.to_csv(output_file_path, index=False)

print(f"Plik 'jedynki.csv' został zapisany w lokalizacji: {output_file_path}")


Plik 'jedynki.csv' został zapisany w lokalizacji: tmp/jedynki.csv


In [ ]:
import pandas as pd

# Load the 'Database to calculate popularity Poland.csv' file
# encoding='ISO-8859-1',
# sep=',',
popularity_db_path = 'poland/Database to calculate popularity Poland.csv' # Replace with actual path
popularity_db_poland = pd.read_csv(popularity_db_path, low_memory=False)

# Load the 'jedynki.csv' file to get the list of URIs
jedynki_path = 'tmp/jedynki.csv' # Replace with actual path
jedynki_uris = pd.read_csv(jedynki_path)

# Loop through each URI in 'jedynki.csv'
for uri in jedynki_uris['Uri']:
    # Filter records in the popularity database for this URI
    filtered_data = popularity_db_poland[popularity_db_poland['uri'] == uri].copy()

    # Convert 'date' column to datetime for correct sorting using .loc
    filtered_data.loc[:, 'date'] = pd.to_datetime(filtered_data['date'], format='%d/%m/%Y')
    
    

    # Sort the filtered data by 'date' in ascending order
    sorted_filtered_data = filtered_data.sort_values(by='date', ascending=True)

    # Extract the URI part that comes after "https://open.spotify.com/track/"
    uri_part = uri.split('/')[-1]

    # Save the sorted and filtered data to a new CSV file
    output_file_path = f'wyniki_uri/{uri_part}.csv'
    sorted_filtered_data.to_csv(output_file_path, index=False)

    # print(f"File saved: {output_file_path}")

print("All files have been processed and saved.")



In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Ścieżka do katalogu z wynikami
results_directory = 'wyniki_uri/'
images_directory = 'images/'

# Pętla przechodząca przez wszystkie pliki CSV w katalogu wyników
for filename in os.listdir(results_directory):
    if filename.endswith('.csv'):
        # Ścieżka do konkretnego pliku CSV
        file_path = os.path.join(results_directory, filename)

        # Wczytanie danych z pliku CSV
        data = pd.read_csv(file_path)

        # Konwersja kolumny 'date' na format datetime
        data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

        # Rysowanie wykresu liniowego
        plt.figure(figsize=(10, 5))

        # Ustawienie kolorów tła i osi
        plt.gca().set_facecolor('#1a1a1a')
        plt.gca().tick_params(axis='x', colors='lightgray')
        plt.gca().tick_params(axis='y', colors='lightgray')
        plt.gca().spines['bottom'].set_color('white')
        plt.gca().spines['top'].set_color('#1a1a1a')  # Ustawienie na kolor tła, aby ukryć
        plt.gca().spines['right'].set_color('#1a1a1a') # Ustawienie na kolor tła, aby ukryć
        plt.gca().spines['left'].set_color('white')

        # Wykres liniowy
        plt.plot(data['date'], data['position'], marker='o', linestyle='-', color='#49f05b')

        # Obrócenie etykiet daty o 45 stopni i ustawienie ich koloru
        plt.xticks(rotation=45, color='white')
        plt.yticks(color='white')

        # Odwrócenie osi Y, aby pozycja 1 była na górze
        plt.gca().invert_yaxis()

        # Ograniczenie liczby etykiet daty na osi X do 20, jeśli jest ich więcej
        if len(data['date']) > 20:
            plt.gca().xaxis.set_major_locator(plt.MaxNLocator(20))

        plt.ylabel('Pozycja', color='white')

        # Dodanie tytułu z informacjami o utworze
        title = data['title'].iloc[0] if 'title' in data.columns else "Brak tytułu"
        artist = data['artist'].iloc[0] if 'artist' in data.columns else "Brak artysty"
        track = data['track'].iloc[0] if 'track' in data.columns else "Brak ścieżki"
        plt.title(f'Track: {track}\nTitle: {title}\nArtist: {artist}', color='white')

        # Zapisanie wykresu
        plt.savefig(f'{images_directory}/{filename}.png', facecolor='#1a1a1a')

        # Zamykanie wykresu
        plt.close()

print("Wykresy zostały wygenerowane i zapisane.")


Wykresy zostały wygenerowane i zapisane.
